In [32]:
import os
import json
import pandas as pd 
import traceback

In [33]:
from langchain.chat_models import ChatOpenAI


In [34]:
from dotenv  import load_dotenv

load_dotenv()

True

In [39]:
key=os.getenv("OPENAI_API_KEY")

In [40]:
print(key)

In [41]:
llm=ChatOpenAI(openai_api_key=key,model_name="", temperature=0.5)

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [42]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import  get_openai_callback
import PyPDF2

In [19]:
Response_json={
    "1": {
        "no": "1",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    },
    
    "2": {
        "no": "2",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    },
    
    "3": {
        "no": "3",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    }
    
    
}

In [20]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [21]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number"," subject"," tone","response_json"],
    template=TEMPLATE
)

In [22]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

NameError: name 'llm' is not defined

In [23]:
template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [24]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=template)

In [ ]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [31]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

NameError: name 'quiz_chain' is not defined

In [25]:
file_path=r"C:\Users\91939\mcqgen\data.txt"

In [26]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [27]:
json.dumps(Response_json)

'{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [28]:
NUMBER=5
SUBJECT="Data science"
tone="simple"

In [30]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

NameError: name 'generate_evaluate_chain' is not defined

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
quiz_str=response.get("quiz")

In [ ]:
quiz_dict = json.loads(quiz_str)

In [ ]:
quiz_dict

In [ ]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
df=pd.DataFrame(quiz_table_data)